In [1]:
import os
import glob
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split

# TensorFlow ≥2.0 is required<
import tensorflow as tf
from tensorflow import keras
#from keras.optimizers import RMSprop
assert tf.__version__ >= "2.0"

#tf 1.x , keras > 2.2 , kerassurgeon 
#tf.compat.v1.disable_v2_behavior() #shap

In [27]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)





In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
y_train_full

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [5]:
X_train_full[0]#60 000 images de 28*28 pixels

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0,   0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,
          1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          0,  36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,
          0,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,
          0, 102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,
         10,   0],
       [  

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
len(X_train_full)

60000

In [8]:
y_valid

array([9, 0, 0, ..., 5, 8, 3], dtype=uint8)

In [9]:
train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train))
valid_set = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [10]:
train_set

<ShuffleDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>

# TFRECORD format

In [11]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

In [12]:
BytesList = tf.train.BytesList
FloatList = tf.train.FloatList
Int64List = tf.train.Int64List
Feature = tf.train.Feature
Features = tf.train.Features
Example = tf.train.Example

In [13]:
for image, label in train_set.take(1):
    image_ser = tf.io.serialize_tensor(image)
    image_example = Example(
        features=Features(
            feature={
            "image": Feature(bytes_list=BytesList(value=[image_ser.numpy()])),
            "label": Feature(int64_list=Int64List(value=[label]))
            }
        )
        )
image_example

features {
  feature {
    key: "image"
    value {
      bytes_list {
        value: "\010\004\022\010\022\002\010\034\022\002\010\034\"\220\006\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\00

In [14]:
with tf.io.TFRecordWriter("protobufs/mes_images") as f:
    for image, label in train_set:
        image_ser = tf.io.serialize_tensor(image)
        image_example = Example(
        features=Features(
            feature={
            "image": Feature(bytes_list=BytesList(value=[image_ser.numpy()])),
            "label": Feature(int64_list=Int64List(value=[label]))
            }
        )
        )
        f.write(image_example.SerializeToString())

In [15]:
def create_example(image, label):
    image_data = tf.io.serialize_tensor(image)
    #image_data = tf.io.encode_jpeg(image[..., np.newaxis])
    return Example(
        features=Features(
            feature={
                "image": Feature(bytes_list=BytesList(value=[image_data.numpy()])),
                "label": Feature(int64_list=Int64List(value=[label])),
            }))

In [16]:
for image, label in train_set.take(1):
    print(create_example(image, label))

features {
  feature {
    key: "image"
    value {
      bytes_list {
        value: "\010\004\022\010\022\002\010\034\022\002\010\034\"\220\006\000\000\000\000\000\000\000\000\000\000\006\000\000\210\250d\000\000\001\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\003\000\016\276sE\262\220\000\004\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\001\000\000\240+\000\000\000\225)\000\005\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\006\000p~\000\005\003\000\n\233\000\003\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\262\000\000\000\000\000\000\220\005\000\001\000\000\000\000\000\000\000\000\000\000\000\001\004\004\007\000D\321\000\000\004\004\000\000\356G\000\004\003\002\003\000\000\000\000\000\000\000\000\000\000\000\000\000W\325\000\000\000\000\000\000\326r\000\000\000\000\000\000\000\000\000\000\000\000\000DlbhQ\251\326XS]YZ<\266\252;UKM\031\000\000\000\000\000\000\000\000\251\347\354\361\366\33

In [17]:
feature_descriptions = {
        "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "label": tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }

In [18]:
from keras import Input

def parse_examples(serialized_examples):
    examples = tf.io.parse_example(serialized_examples, feature_descriptions)
    #targets = examples.pop("label") # separate the targets
    image = tf.io.parse_tensor(examples["image"], out_type=tf.uint8)
    print(image)
    image = tf.reshape(image, shape=[28, 28], name="image")
    print(image)
    print(examples["label"])
    examples["image"] = image
    return image, examples["label"]  #examples #et non return image, examples["label"] sinon on ne peut pas accéder aux noms des tensors...

batch_size = 32
train_set = tf.data.TFRecordDataset(["protobufs/mes_images"])
train_set = train_set.shuffle(60000)
print("0", train_set)
train_set = train_set.map(parse_examples)
print("1", train_set)
train_set = train_set.batch(batch_size)
print("2", train_set)
train_set = train_set.prefetch(1)
print("3", train_set)

0 <ShuffleDataset shapes: (), types: tf.string>
Tensor("ParseTensor:0", dtype=uint8)
Tensor("image:0", shape=(28, 28), dtype=uint8)
Tensor("ParseExample/ParseExampleV2:1", shape=(), dtype=int64)
1 <MapDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.int64)>
2 <BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.uint8, tf.int64)>
3 <PrefetchDataset shapes: ((None, 28, 28), (None,)), types: (tf.uint8, tf.int64)>


In [19]:
train_set

<PrefetchDataset shapes: ((None, 28, 28), (None,)), types: (tf.uint8, tf.int64)>

In [36]:
for image, label in train_set.take(2):
    print(image)
    print(label)

tf.Tensor(
[[[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 ...

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 1  0  0 ... 61 29  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]]]

# Models

In [31]:

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])



model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam", metrics=["accuracy"])
model.fit(train_set, validation_data=(X_valid, y_valid),  epochs=5)

Epoch 1/5
1719/1719 [==============================] - 3s 1ms/step - loss: 3.0160 - accuracy: 0.6623 - val_loss: 0.8262 - val_accuracy: 0.6810
Epoch 2/5
1719/1719 [==============================] - 2s 1ms/step - loss: 0.7191 - accuracy: 0.7126 - val_loss: 0.7908 - val_accuracy: 0.7708
Epoch 3/5
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6271 - accuracy: 0.7735 - val_loss: 0.5726 - val_accuracy: 0.8100
Epoch 4/5
1719/1719 [==============================] - 3s 1ms/step - loss: 0.5590 - accuracy: 0.8023 - val_loss: 0.5679 - val_accuracy: 0.8070
Epoch 5/5
1719/1719 [==============================] - 3s 1ms/step - loss: 0.5167 - accuracy: 0.8199 - val_loss: 0.5169 - val_accuracy: 0.8292


In [23]:

from keras.models import Model

input_image = Input(shape=(28,28,), dtype='uint8')
x= keras.layers.Flatten()(input_image)
x =keras.layers.Dense(100, activation="relu")(x)
output = keras.layers.Dense(10, activation="softmax")(x)

model = Model(input_image, output)
model.summary()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam", metrics=["accuracy"])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(train_set, validation_data=(X_valid, y_valid), epochs=5)

Epoch 1/5
1719/1719 [==============================] - 3s 1ms/step - loss: 3.6214 - accuracy: 0.7065 - val_loss: 0.9087 - val_accuracy: 0.7380
Epoch 2/5
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6986 - accuracy: 0.7554 - val_loss: 0.6517 - val_accuracy: 0.7780
Epoch 3/5
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5866 - accuracy: 0.7835 - val_loss: 0.5729 - val_accuracy: 0.8026
Epoch 4/5
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5222 - accuracy: 0.8103 - val_loss: 0.5286 - val_accuracy: 0.8236
Epoch 5/5
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4882 - accuracy: 0.8289 - val_loss: 0.4993 - val_accuracy: 0.8444
